In [93]:
import os
import sys
module_path = os.path.abspath(os.path.join('fede'))
if module_path not in sys.path:
    sys.path.append(module_path)

from client import Client
import pandas as pd
from supported_modles import Supported_modles
import utils
from fedavg import Fedavg
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import random

In [94]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [95]:
clients = utils.set_data(True)

Creating node1.
Creating node2.
Creating node3.
Creating node4.
Creating node5.
DOWNSAMLING node2
DOWNSAMLING node4
DOWNSAMLING node5


In [96]:
selected_model = Supported_modles.logistic_regression

In [97]:
f1_before = []
for client in clients:
    client.split_data()
    # client.downsample_data(['sbytes','dbytes','sttl','dttl','spkts','dpkts'])
    # client.prep_data()
    client.train_model(selected_model)
    f1_before.append(client.f1)

In [98]:
f1_before

[0.8044233807266984,
 0.9162861491628614,
 0.8668208503824943,
 0.9771689497716894,
 0.9945298254753843]

In [99]:
selected_model = Supported_modles.SGD_classifier

In [100]:
# clients = utils.set_data(True)
for client in clients:
    # client.split_data()
    client.init_empty_model(selected_model,None)

In [101]:
fedavg = Fedavg("global", 0.1)
fedavg.init_global_model(selected_model, None,78)

[Errno 98] Address already in use
Waitiing for a Connection..


In [102]:
number_of_rounds = 40
batch_size = 0.2
epochs = 20
prep = StandardScaler() 

for _ in range(number_of_rounds):
    print(f'Starting new round!')

    applicable_clients = random.sample((clients), random.randint(1, 4))
    applicable_models = []
    applicable_name = []
    round_weights = []
    dataset_size = 0
    

    for client in applicable_clients:
        # print(f'Client name: {client.name}')

        X = client.x
        y = client.y

        X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=batch_size, stratify=y, random_state=random.randint(0,10))
       
        # X_train = prep.fit_transform(X_train)

        dataset_size += X_train.shape[0]
        sample_weights = compute_sample_weight('balanced', y=y_train)

        fedavg.load_global_model(client.model, selected_model)
        fedavg.train_local_agent(X_train, y_train, client.model, epochs, sample_weights, selected_model)
        round_weights.append(X_train.shape[0])
        applicable_models.append(client.model)


    round_weights = np.array(round_weights) / dataset_size # calculate weight based on actual dataset size
    print(round_weights)
    # round_weights = weights
    fedavg.update_global_model(applicable_models, round_weights, selected_model)

Starting new round!
[1.]
Starting new round!
[0.4602559  0.00379848 0.48504598 0.05089964]
Starting new round!
[0.88644501 0.11355499]
Starting new round!
[0.00818542 0.99181458]
Starting new round!
[1.]
Starting new round!
[0.88105626 0.11894374]
Starting new round!
[1.]
Starting new round!
[1.]
Starting new round!
[0.4602559  0.05089964 0.48504598 0.00379848]
Starting new round!
[1.]
Starting new round!
[0.88033382 0.00689405 0.11277213]
Starting new round!
[0.45514215 0.47965679 0.00375628 0.06144478]
Starting new round!
[0.43566051 0.03251198 0.53182752]
Starting new round!
[0.45030067 0.54969933]
Starting new round!
[1.]
Starting new round!
[0.87469605 0.00721884 0.11808511]
Starting new round!
[0.05109372 0.46201084 0.48689544]
Starting new round!
[1.]
Starting new round!
[0.45030067 0.54969933]
Starting new round!
[1.]
Starting new round!
[1.]
Starting new round!
[0.00379848 0.48504598 0.05089964 0.4602559 ]
Starting new round!
[0.00737635 0.09884308 0.89378057]
Starting new rou

In [103]:
f1_fedavg = []
for client in clients:
    # x = prep.transform(client.x_test)
    y_hat = fedavg.model.predict(client.x_test)
    f1_fedavg.append(f1_score(client.y_test,y_hat))
print(f1_fedavg)

[0.8754461493174376, 0.8275669642857143, 0.7901105075856901, 0.5810055865921787, 0.506726457399103]


In [104]:
i = 0
for client in clients:
    print(f'Difference {f1_fedavg[i]-f1_before[i]}')
    i += 1

Difference 0.0710227685907392
Difference -0.08871918487714714
Difference -0.07671034279680422
Difference -0.39616336317951073
Difference -0.4878033680762812
